# An introduction to Gradual Typing in Python

Gradual typing is a programming language feature that allows developers to gradually add type annotations to their code. In Python, this means you can mix statically typed and dynamically typed code within the same program, taking advantage of both worlds. Type annotations can be added to variables, function parameters, and function return values, offering better documentation and enabling more robust static analysis through tools like [mypy](https://mypy-lang.org/). The type system is designed to be flexible and can accommodate a wide range of patterns, including 
+ *generic types*, 
+ *union types*, and 
+ *user defined types*. 

By incrementally adding type annotations, developers can make their Python code more self-explanatory, easier to debug, and more maintainable, while also catching potential type errors before runtime. 

## Installing `mypy`for Jupyter Notebooks
In its full generality, gradual typing is only available as of Python version `3.11`.  Therefore, to use gradual typing in a *conda environment*, you have to create this environment using the following command:
```
conda create -n fl python=3.11 jupyter notebook
```
After activation this environment via the command
```
conda activate fl
```
we can install `mypy` with the command
```
pip install mypy
```
To use `mypy`in a jupyter notebook we have to install the jupyter notebook extension 
[nb mypy](https://pypi.org/project/nb-mypy/) via the command
```
pip install nb_mypy
```
The next cell activates this extension.

!pip install mypy

!pip install nb_mypy

In [ ]:
%load_ext nb_mypy

## Finding Errors by Type Checking

The following two lines contain an error that `mypy` is able to find.

In [ ]:
number = input("What is your favourite number? ")
print("It is", number + 1)  

The correct version of the two lines above would have been as follows:

In [ ]:
number = int(input("What is your favourite number? "))
print(f'It is {number + 1}.') 

## Type Annotations

The most basic form of type checking in Python is specifying the types of variables and function return values.
To type check a function, you annotate the type of a parameter by putting a colon after the name of the variable. 
The return type of the function is specified using the `->` syntax as shown below.

In [ ]:
def add(a: int, b: int) -> int:
    return a + b

In the next cell the **type checker** tells us, that we have called the function `add` with strings instead of integers.  
The **Python interpreter** executes this cell without encountering an error, since the interpreter does not care about the type annotations. 

In [ ]:
name = 'Karl'
add('Hello ', name)    

If necessary, we can inspect the type annotations of a function at runtime via the attribute `__annotations__` as shown below.

In [ ]:
add.__annotations__

## Built-in Types

`mypy` supports all built-in Python types like `int`, `float`, `str`, and `bool`.
Complex types like `list`, `tuple`, and `dict` are also supported.  

The function `average(L)` computes the arithmetic mean of the numbers in the list `L`. 

In [ ]:
def average(numbers: list[int]) -> float:
    return sum(numbers) / len(numbers)

In [ ]:
average([1, 2, 3, 4])

The following cell has a type error, although it executes without a problem.

In [ ]:
average([1.0, 2.0, 3.0, 4.0])

## Custom Types
You can define your own types using the `class` keyword. Note that `self` does not have a type annotation.

In [ ]:
class Person:
    def __init__(self, name: str):
        self.name = name

    def greet(self) -> str:
        return f"Hello, {self.name}!"

When a function does not return a value, the return type is `None`.

In [ ]:
def salve(p: Person) -> None:
    print(p.greet())

In [ ]:
jc = Person('Julius Caesar')
salve(jc)

The function `greet_name` either accepts a string representing a name as its argument, 
or it accepts a dictionary as its argument.  The dictionary is supposed to store both 
the first name under the key `given` and the last name under the key `family`.  

The *union* operator `|` can be used to express the fact that `name`can either be a
`str` or a `dict[str, str]`.

In [ ]:
def greet_name(name: str | dict[str, str]) -> str:
    if isinstance(name, str):
        return 'Hi ' + name + '!'
    if isinstance(name, dict):
        return f"Bienvenido, Señor {name['given']} {name['family']}."

In [ ]:
greet_name("Alice")

In [ ]:
greet_name({'given': 'Esteban', 'family': 'Ramirez'})

## Using `TypeVar` for Generic Functions

In [ ]:
from typing import TypeVar

The next example shows how to type *generic*  functions.  This is done using the function `TypeVar`, 
which creates a new type variable.

In [ ]:
S = TypeVar('S')
T = TypeVar('T')

The function `swap` takes a pair of elements that should be of the same type.  
It swaps the order of these elements. swaps the elements of a pair (a 2-tuple). The function   
`swap` is *generic*, meaning it is able to handle pairs of integers, strings, or any other type.

In [ ]:
def swap(pair: tuple[S, T]) -> tuple[T, S]:
    x, y = pair
    return y, x

In the next cell, the type variable `T` is instantiated as `int`.

In [ ]:
swap((1, 2))

In the following cell, the type variable `T` is instantiated as `str`.

In [ ]:
swap(('a', 'b'))

Below, the type variable the type variable `T` is instantiated as `object`.

In [ ]:
swap((1, 'a'))

## Recursive Types

In [ ]:
RecursiveTuple = TypeVar("RecursiveTuple")
RecursiveTuple = int | str | tuple[RecursiveTuple, ...]

In [ ]:
def flatten_recursive_tuple(t: RecursiveTuple) -> list[int | str]:
    result = []
    if isinstance(t, (int, str)):
        result.append(t)
    elif isinstance(t, tuple):
        for elem in t:
            result.extend(flatten_recursive_tuple(elem))
    return result

In [ ]:
nested_tuple: RecursiveTuple = (1, "a", (2, "b", (3, "c")))
flattened = flatten_recursive_tuple(nested_tuple)
print(flattened) 